# 結果の確認  
グリッドサーチ後の各モデルの精度を比較

In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, precision_score, recall_score
from keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns

Using TensorFlow backend.


Load Validation Dataset

In [2]:
df = pd.read_hdf('data.h5', key='test_selected', mode='r')
X, Y = np.array(df.drop('Class', axis=1)), np.array(df['Class'])

Load Training Dataset

In [3]:
df_t = pd.read_hdf('data.h5', key='train_selected', mode='r')
X_t, Y_t = np.array(df_t.drop('Class', axis=1)), np.array(df_t['Class'])

Load Model

In [4]:
lr = joblib.load('lr.pkl') 
mlp = joblib.load('mlp.pkl') 
dt = joblib.load('dt.pkl') 
svm = joblib.load('svm.pkl')
xgb = joblib.load('xgb.pkl')
rf = joblib.load('rf.pkl')
dnn_ae3=load_model('DNN_ae3.h5')
dnn_ae4=load_model('DNN_ae4.h5')
dnn_ae5=load_model('DNN_ae5.h5')

/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Compile DNN

In [5]:
dnn_ae3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
dnn_ae4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
dnn_ae5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

各モデルの加重平均スコアを集計

In [6]:
model=[lr, mlp, dt, svm, xgb, rf]
model_dnn=[dnn_ae3, dnn_ae4, dnn_ae5]
model_name= ['lr', 'mlp', 'dt', 'svm', 'xgb', 'rf']
model_name_dnn = ['dnn3', 'dnn4', 'dnn5']
dnn_map = {0:-1, 1:0, 2:1}

result_weighted={}
result_m={}
result_z={}
result_p={}
for s, t in zip(model, model_name):
    res_model = {}
    res_model['F1']=f1_score(Y, s.predict(X), average='weighted')
    res_model['Precision Score'] = precision_score(Y, s.predict(X), average='weighted')
    res_model['Recall Score'] = recall_score(Y, s.predict(X), average='weighted')
    
    res_model['Train F1']=f1_score(Y_t, s.predict(X_t), average='weighted')
    res_model['Train Precision Score'] = precision_score(Y_t, s.predict(X_t), average='weighted')
    res_model['Train Recall Score'] = recall_score(Y_t, s.predict(X_t), average='weighted')

    result_weighted[t] = res_model
    
for s, t in zip(model_dnn, model_name_dnn):
    res_model = {}
    res_model['F1']=f1_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average='weighted')
    res_model['Precision Score'] = precision_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average='weighted')
    res_model['Recall Score'] = recall_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average='weighted')

    res_model['Train F1']=f1_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average='weighted')
    res_model['Train Precision Score'] = precision_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average='weighted')
    res_model['Train Recall Score'] = recall_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average='weighted')

    result_weighted[t] = res_model

/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 992/1610 [=================>............] - ETA: 0s

各モデルの-1ラベルのスコアを集計

In [7]:
for s, t in zip(model, model_name):
    res_model = {}
    res_model['F1']=f1_score(Y, s.predict(X), average=None)[0]
    res_model['Precision Score'] = precision_score(Y, s.predict(X), average=None)[0]
    res_model['Recall Score'] = recall_score(Y, s.predict(X), average=None)[0]
    
    res_model['Train F1']=f1_score(Y_t, s.predict(X_t), average=None)[0]
    res_model['Train Precision Score'] = precision_score(Y_t, s.predict(X_t), average=None)[0]
    res_model['Train Recall Score'] = recall_score(Y_t, s.predict(X_t), average=None)[0]

    result_m[t] = res_model
    
for s, t in zip(model_dnn, model_name_dnn):
    res_model = {}
    res_model['F1']=f1_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[0]
    res_model['Precision Score'] = precision_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[0]
    res_model['Recall Score'] = recall_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[0]

    res_model['Train F1']=f1_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[0]
    res_model['Train Precision Score'] = precision_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[0]
    res_model['Train Recall Score'] = recall_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[0]

    result_m[t] = res_model

/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 960/1610 [================>.............] - ETA: 0s

各モデルの0ラベルのスコアを集計

In [8]:
for s, t in zip(model, model_name):
    res_model = {}
    res_model['F1']=f1_score(Y, s.predict(X), average=None)[1]
    res_model['Precision Score'] = precision_score(Y, s.predict(X), average=None)[1]
    res_model['Recall Score'] = recall_score(Y, s.predict(X), average=None)[1]
    
    res_model['Train F1']=f1_score(Y_t, s.predict(X_t), average=None)[1]
    res_model['Train Precision Score'] = precision_score(Y_t, s.predict(X_t), average=None)[1]
    res_model['Train Recall Score'] = recall_score(Y_t, s.predict(X_t), average=None)[1]

    result_z[t] = res_model
    
for s, t in zip(model_dnn, model_name_dnn):
    res_model = {}
    res_model['F1']=f1_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[1]
    res_model['Precision Score'] = precision_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[1]
    res_model['Recall Score'] = recall_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[1]

    res_model['Train F1']=f1_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[1]
    res_model['Train Precision Score'] = precision_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[1]
    res_model['Train Recall Score'] = recall_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[1]

    result_z[t] = res_model

/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 896/1610 [===============>..............] - ETA: 0s

各モデルの+1ラベルのスコアを集計

In [9]:
for s, t in zip(model, model_name):
    res_model = {}
    res_model['F1']=f1_score(Y, s.predict(X), average=None)[2]
    res_model['Precision Score'] = precision_score(Y, s.predict(X), average=None)[2]
    res_model['Recall Score'] = recall_score(Y, s.predict(X), average=None)[2]
    
    res_model['Train F1']=f1_score(Y_t, s.predict(X_t), average=None)[2]
    res_model['Train Precision Score'] = precision_score(Y_t, s.predict(X_t), average=None)[2]
    res_model['Train Recall Score'] = recall_score(Y_t, s.predict(X_t), average=None)[2]

    result_p[t] = res_model
    
for s, t in zip(model_dnn, model_name_dnn):
    res_model = {}
    res_model['F1']=f1_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[2]
    res_model['Precision Score'] = precision_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[2]
    res_model['Recall Score'] = recall_score(Y, pd.Series(s.predict_classes(X)).map(dnn_map), average=None)[2]

    res_model['Train F1']=f1_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[2]
    res_model['Train Precision Score'] = precision_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[2]
    res_model['Train Recall Score'] = recall_score(Y_t, pd.Series(s.predict_classes(X_t)).map(dnn_map), average=None)[2]

    result_p[t] = res_model

/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 928/1610 [================>.............] - ETA: 0s

結果を保存

In [10]:
result_m = pd.DataFrame(result_m)
result_z = pd.DataFrame(result_z)
result_p = pd.DataFrame(result_p)
result_weighted = pd.DataFrame(result_weighted)
result_m

,dnn3,dnn4,dnn5,dt,lr,mlp,rf,svm,xgb
F1,0.156250,0.083333,0.0,0.097561,0.0,0.108108,0.092308,0.179487,0.036364
Precision Score,0.500000,0.166667,0.0,0.142857,0.0,0.200000,0.272727,0.291667,1.000000
Recall Score,0.092593,0.055556,0.0,0.074074,0.0,0.074074,0.055556,0.129630,0.018519
Train F1,1.000000,1.000000,1.0,0.565789,0.0,1.000000,0.968825,0.997658,0.876640
Train Precision Score,1.000000,1.000000,1.0,0.955556,0.0,1.000000,0.995074,1.000000,1.000000
Train Recall Score,1.000000,1.000000,1.0,0.401869,0.0,1.000000,0.943925,0.995327,0.780374


In [11]:
result_z

,dnn3,dnn4,dnn5,dt,lr,mlp,rf,svm,xgb
F1,0.814696,0.812903,0.815287,0.779832,0.824261,0.782178,0.814696,0.800000,0.823161
Precision Score,0.706371,0.709859,0.705234,0.703030,0.701058,0.695015,0.706371,0.711765,0.703209
Recall Score,0.962264,0.950943,0.966038,0.875472,1.000000,0.894340,0.962264,0.913208,0.992453
Train F1,1.000000,1.000000,1.000000,0.896816,0.842975,1.000000,0.990283,0.999574,0.962659
Train Precision Score,1.000000,1.000000,1.000000,0.815202,0.728571,1.000000,0.981575,0.999148,0.928006
Train Recall Score,1.000000,1.000000,1.000000,0.996590,1.000000,1.000000,0.999147,1.000000,1.000000


In [12]:
result_p

,dnn3,dnn4,dnn5,dt,lr,mlp,rf,svm,xgb
F1,0.030303,0.031250,0.057971,0.075949,0.0,0.052632,0.000000,0.082192,0.032258
Precision Score,0.142857,0.200000,0.200000,0.150000,0.0,0.117647,0.000000,0.214286,0.333333
Recall Score,0.016949,0.016949,0.033898,0.050847,0.0,0.033898,0.000000,0.050847,0.016949
Train F1,1.000000,1.000000,1.000000,0.543689,0.0,1.000000,0.967890,1.000000,0.890547
Train Precision Score,1.000000,1.000000,1.000000,0.976744,0.0,1.000000,0.990610,1.000000,1.000000
Train Recall Score,1.000000,1.000000,1.000000,0.376682,0.0,1.000000,0.946188,1.000000,0.802691


In [13]:
result_weighted

,dnn3,dnn4,dnn5,dt,lr,mlp,rf,svm,xgb
F1,0.598201,0.586675,0.580612,0.572499,0.577855,0.572011,0.584336,0.599316,0.587314
Precision Score,0.588934,0.552679,0.525627,0.536686,0.491483,0.534180,0.534168,0.574102,0.687875
Recall Score,0.690476,0.677249,0.682540,0.632275,0.701058,0.642857,0.682540,0.666667,0.701058
Train F1,1.000000,1.000000,1.000000,0.803905,0.614168,1.000000,0.984329,0.999378,0.941237
Train Precision Score,1.000000,1.000000,1.000000,0.856233,0.530816,1.000000,0.984620,0.999379,0.947547
Train Recall Score,1.000000,1.000000,1.000000,0.831677,0.728571,1.000000,0.984472,0.999379,0.943478


In [14]:
result_m.to_csv('result_m.csv')
result_z.to_csv('result_z.csv')
result_p.to_csv('result_p.csv')
result_weighted.to_csv('result_weighted.csv')

各モデルの予測値を保存

In [15]:
predict = {}
predict_t = {}
predict['Act'] = Y
predict_t['Act'] = Y_t

for s, t in zip(model, model_name):
    predict[t]=s.predict(X)
    predict_t[t]=s.predict(X_t)

for s, t in zip(model_dnn, model_name_dnn):
    predict[t]= list(pd.Series(s.predict_classes(X)).map(dnn_map))
    predict_t[t]= list(pd.Series(s.predict_classes(X_t)).map(dnn_map))

predict = pd.DataFrame(predict, index = df.index)
predict_t = pd.DataFrame(predict_t, index = df_t.index)
predict.to_csv('predict_test.csv')
predict_t.to_csv('predict_train.csv')
predict.head()

 992/1610 [=================>............] - ETA: 0s

,Act,dnn3,dnn4,dnn5,dt,lr,mlp,rf,svm,xgb
2015-01-02,0,0,0,0,0,0,0,0,0,0
2015-01-05,-1,0,0,0,0,0,0,0,0,0
2015-01-06,0,0,0,0,0,0,0,0,0,0
2015-01-07,1,0,0,0,0,0,0,0,0,0
2015-01-08,1,0,0,0,0,0,0,0,0,0


In [16]:
predict_t.head()

,Act,dnn3,dnn4,dnn5,dt,lr,mlp,rf,svm,xgb
2008-08-11,0,0,0,0,0,0,0,0,0,0
2008-08-12,-1,-1,-1,-1,0,0,-1,-1,-1,-1
2008-08-13,0,0,0,0,0,0,0,0,0,0
2008-08-14,0,0,0,0,0,0,0,0,0,0
2008-08-15,0,0,0,0,0,0,0,0,0,0
